### UC Berkeley, MICS, W202-Cryptography
### Week 06 Breakout 6
### Elliptic Curve - Prime Curves - Attacks: too few points, too short a period for scalar multiplication, too small a scalar multiplier, Invalid Curve Attacks

There are several attacks that can be made on Elliptic Curve Prime Curves:

1) Too few points - our choice for p determines the number of points

2) Too short a period for scalar multiplication - our choices for A and B determine the period for scalar multiplication. A common misconception is that a large p is all we need for a secure prime curve.

3) Too small a scalar multiplier

4) Invalid Curve Attacks

For 1 and 2, an attack is pretty simple.  We just start with the base point and 1 for the scalar multiplier. We keep iterating adding 1 to the scalar multiplier.  We keep a list of the scalar multipliers and for each the point it generates. Once we get back to the base point, we know we have been through the entire period, and we now have a table of ellipitc curve discrete logarithms for every point.  (We did this in this week's breakout 1)

For 3, an attack is also very simple.  We just start with the base point and 1 for the scalar multiplier. We keep adding 1 to the scalar multiplier until we get to the point.  The scalar multiplier that gets us to that point will be the elliptic curve discrete logarithm for that point.

**For 4, Invalid Curve Attacks, this is much more difficult and will be the subject of this breakout**

* A, B, p are publically known

* Think about the math behind scalar multiplication:  We use A and p but NOT B. A is used for the slope calculation for adding a point to itself. p is used in all calculations.

* If we are given a prime curve, we cannot alter the value of A nor p.  But we can alter the value for B since it's not used in calculations. 

* Recall that A and B determine the scalar period.  We cannot alter A, but we can alter B to create a second "invalid curve" with a really short period of say 5.  

* Note: if we have two prime curves that have the same A and p values, but not the same B value, we say the two prime curves "lie on the same elliptic plane"

* In the Diffie Hellman Elliptic Curve Key Exchange, suppose Evil Bob gives Alice a point from the invalid curve with a period of 5?  

* If Alice validates the point from the invalid curve, she will know it's an invalid curve attack and end the key exchange.  

* However if Alice does not validate the point from the invalid curve, and establishes a key with Evil Bob, Evil Bob will learn some insights into Alice's secret scalar multiplier k. 

* Evil Bob can continue to submit more points on an invalid curve to Alice, he can gain even more insights.  Evil Bob can also submit points from other invalid curves with periods of 7, 11, 13, 17, etc. (prime numbers for the periods) and use the Chinese Remainder Theorem to combine them to recover Alice's secret scalar multiplier k.

In 2018, there was a series of attacks on routers using an Invalid Curve Attack on the Diffie Hellman Elliptic Curve Key Exchange.  Apparently the routers were not validating points.  Of course if they had simply validated points, the attacks would not have been successful.

Ordinarily, in a FIPS curve, it's almost impossible to hit the point at INFINITY. If we do so, the proper way to handle it is to go to the next point if we hit INFINITY.  With invalid curves, the give away was that some routers were hitting INFINITY and dumping core.  Eventually the core dumps got noticed.  If the core dumps had not been noticed, the invalid curve attacks might have gone unnoticed.  

Also, we have no idea how long the invalid curve attacks had been going on for and what data was stolen as a result!

In [1]:
from sage.all import *

In [2]:
def my_print_number(label, x):
    "prints a number in decimal, number of digits, hex, number of bits"
    
    print ("\n", label, '\n')
    print ("decimal:", "{:,}".format(x), "\n")
    print ("number of digits:", x.ndigits(), "\n")
    print ("hex:", x.hex(), "\n")
    print ("number of bits:", x.nbits(), "\n")

#### We will start with the standard NIST-256 curve (same curve used in the 2018 invalid curve attacks)

In [3]:
# We will use the prime curve used in the Summer 2018 attacks

# NIST-256 aka NIST P-256 aka secp256r1

p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff

A = -3
B = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b

P_x = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
P_y = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5

E = EllipticCurve(GF(p), [A,B])

P = E(P_x,P_y)

In [4]:
my_print_number("A", A)


 A 

decimal: -3 

number of digits: 1 

hex: -3 

number of bits: 2 



In [5]:
my_print_number("B", B)


 B 

decimal: 41,058,363,725,152,142,129,326,129,780,047,268,409,114,441,015,993,725,554,835,256,314,039,467,401,291 

number of digits: 77 

hex: 5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b 

number of bits: 255 



In [6]:
my_print_number("p", p)


 p 

decimal: 115,792,089,210,356,248,762,697,446,949,407,573,530,086,143,415,290,314,195,533,631,308,867,097,853,951 

number of digits: 78 

hex: ffffffff00000001000000000000000000000000ffffffffffffffffffffffff 

number of bits: 256 



In [7]:
my_print_number("Px", P_x)


 Px 

decimal: 48,439,561,293,906,451,759,052,585,252,797,914,202,762,949,526,041,747,995,844,080,717,082,404,635,286 

number of digits: 77 

hex: 6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296 

number of bits: 255 



In [8]:
my_print_number("Py", P_y)


 Py 

decimal: 36,134,250,956,749,795,798,585,127,919,587,881,956,611,106,672,985,015,071,877,198,253,568,414,405,109 

number of digits: 77 

hex: 4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5 

number of bits: 255 



#### Create an invalid curve with the same A and p as NIST-256 but with a different B which will have a very short scalar multiplication period of only 5 points.  We will also define our own base point.

In [9]:
invalid_B = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604f

invalid_E = EllipticCurve(GF(p), [A, invalid_B])

invalid_P_x = 0xb70bf043c144935756f8f4578c369cf960ee510a5a0f90e93a373a21f0d1397f
invalid_P_y = 0x4a2e0ded57a5156bb82eb4314c37fd4155395a7e51988af289cce531b9c17192

invalid_P = invalid_E(invalid_P_x, invalid_P_y)

In [10]:
my_print_number("invalid B", invalid_B)


 invalid B 

decimal: 41,058,363,725,152,142,129,326,129,780,047,268,409,114,441,015,993,725,554,835,256,314,039,467,401,295 

number of digits: 77 

hex: 5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604f 

number of bits: 255 



In [11]:
my_print_number("invalid Px", invalid_P_x)


 invalid Px 

decimal: 82,794,344,854,243,450,371,984,501,721,340,198,645,022,926,339,504,713,863,786,955,730,156,937,886,079 

number of digits: 77 

hex: b70bf043c144935756f8f4578c369cf960ee510a5a0f90e93a373a21f0d1397f 

number of bits: 256 



In [12]:
my_print_number("invalid Py", invalid_P_y)


 invalid Py 

decimal: 33,552,521,881,581,467,670,836,617,859,178,523,407,344,471,948,513,881,718,969,729,275,859,461,829,010 

number of digits: 77 

hex: 4a2e0ded57a5156bb82eb4314c37fd4155395a7e51988af289cce531b9c17192 

number of bits: 255 



In [13]:
def my_scalar_multiplication(P, k):
    "given a base point P in a SageMath elliptic prime curve, and a scalar multiplier k, loop from 1 to k showing scalar multiplication"
    
    for i in range(1, k+1):
        
        print (str(i) + " * P = ")
        
        Q = i * P
        
        if Q[2] == 0:
            print ("          INFINITY")
        else:
            Q_x = Integer(Q[0])
            Q_y = Integer(Q[1])
            print ("          " + str(Q_x))
            print ("          " + str(Q_y))
        

#### The scalar period of the "invalid curve" is 5.  If someone accepts a point on this curve without validating it, when they perform scalar multiplication, they will get one of these 5 points. 

#### Ordinarily, in a FIPS curve, it's almost impossible to hit the point at INFINITY. If we do so, the proper way to handle it is to go to the next point if we hit INFINITY.  With invalid curves, the give away was that some routers were hitting INFINITY and dumping core.  Eventually the core dumps got noticed.  If the core dumps had not been noticed, the invalid curve attacks might have gone unnoticed.  

In [14]:
my_scalar_multiplication(invalid_P, 15)

1 * P = 
          82794344854243450371984501721340198645022926339504713863786955730156937886079
          33552521881581467670836617859178523407344471948513881718969729275859461829010
2 * P = 
          46111711714004764615393195350570532019484583409650937480110926637425134418118
          58716222405328743118080845934227278038278303558676945382860804917761871042597
3 * P = 
          46111711714004764615393195350570532019484583409650937480110926637425134418118
          57075866805027505644616601015180295491807839856613368812672826391105226811354
4 * P = 
          82794344854243450371984501721340198645022926339504713863786955730156937886079
          82239567328774781091860829090229050122741671466776432476563902033007636024941
5 * P = 
          INFINITY
6 * P = 
          82794344854243450371984501721340198645022926339504713863786955730156937886079
          33552521881581467670836617859178523407344471948513881718969729275859461829010
7 * P = 
          4611171171400476461539319535

#### Let's revisit Diffie Hellman and see how it would play out if Alice accepts a point on the invalid curve from an Evil Bob

#### Alice has generated a scalar multiplier which she keeps secret.   

In [15]:
alice_scalar = 0xb3feb531f98042ad46ccf3696c183234e4166906879f72d3251d2c893e3ecd4d

In [16]:
my_print_number("Alice's secret scalar multiplier", alice_scalar)


 Alice's secret scalar multiplier 

decimal: 81,414,029,614,260,072,837,856,622,660,195,989,797,637,896,290,832,594,436,659,895,002,551,400,975,693 

number of digits: 77 

hex: b3feb531f98042ad46ccf3696c183234e4166906879f72d3251d2c893e3ecd4d 

number of bits: 256 



#### Alice calculates alice_scalar * P and sends this point to Bob.  Alice also reuses this same point, so everyone who communicates with her gets the same point.  This is safe unless they can solve the Elliptic Curve Discrete Logarithm Problem.

In [17]:
alice_sends = alice_scalar * P

In [18]:
my_print_number("Alice sends Bob - x value of point", Integer(alice_sends[0]))


 Alice sends Bob - x value of point 

decimal: 10,479,250,359,931,507,646,642,743,289,956,409,818,560,536,137,113,993,267,635,475,994,104,003,121,693 

number of digits: 77 

hex: 172b0ba6e581e407c352f0ee21b19a7c08ddc0544e1b9a16ced20b092b35521d 

number of bits: 253 

In [19]:
my_print_number("Alice sends Bob - y value of point", Integer(alice_sends[1]))


 Alice sends Bob - y value of point 

decimal: 106,564,899,867,833,198,877,135,229,405,876,178,560,500,774,851,495,257,726,933,446,012,389,212,213,764 

number of digits: 78 

hex: eb99988c5f1e9bbaba0cadfe892f3d2268e3fd22f045013333a5f7500a9ed204 

number of bits: 256 



#### Evil Bob breaks the standard protocol at this point.  Evil Bob sends Alice the base point on the invalid curve.  

In [20]:
bob_sends = invalid_P

In [21]:
my_print_number("Evil Bob sends Alice the base point on the invalid curve - x value of point", Integer(bob_sends[0]))


 Evil Bob sends Alice the base point on the invalid curve - x value of point 

decimal: 82,794,344,854,243,450,371,984,501,721,340,198,645,022,926,339,504,713,863,786,955,730,156,937,886,079 

number of digits: 77 

hex: b70bf043c144935756f8f4578c369cf960ee510a5a0f90e93a373a21f0d1397f 

number of bits: 256 



In [22]:
my_print_number("Evil Bob sends Alice the base point on the invalid curve - y value of point", Integer(bob_sends[1]))


 Evil Bob sends Alice the base point on the invalid curve - y value of point 

decimal: 33,552,521,881,581,467,670,836,617,859,178,523,407,344,471,948,513,881,718,969,729,275,859,461,829,010 

number of digits: 77 

hex: 4a2e0ded57a5156bb82eb4314c37fd4155395a7e51988af289cce531b9c17192 

number of bits: 255 



#### Alice hasn't taken MICS W202, and got some bad information off the internet, so she doesn't know she needs to validate the point.  Alice accepts the point on the invalid curve.  Alice multipies her secret scalar multiplier by the point she received from Bob and uses it as the key to symmetric AES-256.

In [23]:
alice_key = alice_scalar * bob_sends

In [24]:
my_print_number("Alice's key - x value of point", Integer(alice_key[0]))


 Alice's key - x value of point

 

decimal: 46,111,711,714,004,764,615,393,195,350,570,532,019,484,583,409,650,937,480,110,926,637,425,134,418,118 

number of digits: 77 

hex: 65f24dcf15472651d89b04e2f38f9f5424678815b4fc44cc4e42b93512b038c6 

number of bits: 255 



In [25]:
my_print_number("Alice's key - y value of point", Integer(alice_key[1]))


 Alice's key - y value of point 

decimal: 57,075,866,805,027,505,644,616,601,015,180,295,491,807,839,856,613,368,812,672,826,391,105,226,811,354 

number of digits: 77 

hex: 7e2fcbba1719ffbdaab6e048da94206a8e05f6eb4ba15df328e5d0f5729627da 

number of bits: 255 



#### Note that Alice's key = 3 * P on the invalid curve

#### Evil Bob 

* Evil Bob knows that Alice has only 5 choices for the key. 

* Evil Bob can try all 5 points and see which one works as the key.

* Evil Bob finds out that 3 * P works.  Evil Bob concludes that Alice's secret scalar multiplier mod 5 yields 3.

* Evil Bob can try with other invalid curves with periods of 7, 11, 13, etc.

* Evil Bob can plug the values into the Chinese Remainder Theorem and solve.  
